In [ ]:
import pandas as pd

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="profoz/distilbert-base-cased-finetuned-stars")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# Example text for sentiment analysis
text = "I loved the food at that restaurant, it was amazing!"

# Perform sentiment analysis using the pipeline
result = pipe(text)

print(result)

[{'label': 'LABEL_4', 'score': 0.9521932005882263}]


In [ ]:
# Get model configuration
model_config = pipe.model.config

# Check the number of labels/classes
num_labels = model_config.num_labels
print(f"Number of labels/classes: {num_labels}")

Number of labels/classes: 5


In [ ]:
# Truncate the text to fit within maximum sequence length
max_seq_length = pipe.tokenizer.model_max_length

max_seq_length

512

In [ ]:
test_df = pd.read_csv("/content/test_df.csv")

In [ ]:
print(test_df[10156:10157]['text'])

10156    friend love place much wed reception find head...
Name: text, dtype: object


In [ ]:
test_df.text = test_df.text.astype(str)

In [ ]:
test_df_small = test_df.sample(frac=0.1)

In [ ]:
test_df_small.dtypes

label     int64
text     object
dtype: object

In [ ]:
y_true = test_df_small['label']

In [ ]:
#intializing lists to store pred and labels for batch processing
preds = []

In [ ]:
#iterating through test data set

for index, row in test_df_small.iterrows():

    #performing sentiment analysis on each row
    text = row['text']

    # Truncate the text to fit within maximum sequence length
    truncated_text = text[:max_seq_length]

    # Performing sentiment analysis on truncated text
    result = pipe(truncated_text)

    #appending the result to the list
    preds.append(result[0]['label'])

In [ ]:
# Mapping dictionary from labels to numerical values
label_map = {
    'LABEL_0': 0,
    'LABEL_1': 1,
    'LABEL_2': 2,
    'LABEL_3': 3,
    'LABEL_4': 4
}

# Convert predictions to numerical values using list comprehension
preds = [label_map[label] for label in preds]

In [ ]:
test_df_small

,label,text
16480,1,warn nightclub contain bass know state califor...
27062,3,often pop coffee weekend tend bite quieter res...
9302,2,large airport international god lot walk need ...
19179,0,update someone join yelp way send extremely ab...
8009,2,write review salsa yes pollo quesadilla amaze ...
...,...,...
17849,3,fan nthe first morning lv come breakfast 8oz s...
24537,4,small place 9 10 table seat bar want dine dinn...
3249,2,choose hotel base yelp review room clean welco...
21772,1,require make box brownie mix n large mix bowl ...


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
accuracy = accuracy_score(y_true, preds)

confusion_mat = confusion_matrix(y_true, preds)

classification_rep = classification_report(y_true, preds)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.2869327874445582
Confusion Matrix:
[[329   0   3   0 244]
 [139   0  18   0 426]
 [ 29   0   9   4 579]
 [ 12   0   1   4 625]
 [  9   0   1   0 499]]
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.57      0.60       576
           1       0.00      0.00      0.00       583
           2       0.28      0.01      0.03       621
           3       0.50      0.01      0.01       642
           4       0.21      0.98      0.35       509

    accuracy                           0.29      2931
   macro avg       0.33      0.31      0.20      2931
weighted avg       0.33      0.29      0.19      2931

